## Loan Default Prediction

#### Project Description: 
Develop basic understanding of risk anlytics in banking and financial sercice. It is under the category to **credit default** (credit risk accessment), simply explained as the possibility of a loss for an loan investor due to a borrower’s failure to repay a loan. It analyzes borrower's capability to repay by using machine learning and data analytics to help to reduce the default risk and therefore maximize the investment return. 


#### Aim:
- What are the factors affecting loan default rate ? 
- Predict whether a loan will be paid in full or charged off to help investors increase their yield in P2P investing.

Techinqiue used here : Logistic Regression, Naive Bayes, and SVM classifiers

#### Project Outline

- [1.Import data](#section1)
- [2.Data Preprocessing and EDA](#section2)



Feature Selection I: EDA
4.1 Drop features missing too much data
4.2 Remove "cheat" data
4.3 Inspect the remaining features one by one
Feature Section II: (Linear) correlation between (numerical) features
5.1 Convert target variable to 0/1 indicator
5.2 K-S test
5.3 Pearson Correlation
Feature Engineering
6.1 Dummy variables
6.2 Train/Test data split
6.3 Imputing missing values
6.4 Feature Scaling
6.5 PCA visualization of the train data
6.6 Linear dependence of Charged-Off
Modeling
7.1 Logistic Regression with SGD
7.2 Random Forest
7.3 KNN
7.4 Hyperparameter tuning on the best model
7.5 Evaluate the best model on the testing set
Further understanding the model
Conclusion


In [11]:
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
import random
import os, sys
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)


In [12]:
####  stdout style   #####
import logging

logging.basicConfig(
    level=logging.INFO, 
    format='%(levelname)s - - - - - - - - %(message)s  - - - - - - - ',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger('LOGGER_NAME')
# logger.info('title style') # Test

# def groupby_factor(data, factor):
#     df = data.groupby([factor,'loan_status']).agg({factor:  'count'})
#     df = df.unstack(level=1)
#     df.columns = df.columns.get_level_values(1)
#     return (df.div(df.sum(axis=1), axis=0)*100).round(2)

<a id='section1'></a>

### 1 Data Preparation

#### 1.1 Import Data :
Source: Kaggle.

Data Size: 1.8G


Data Desscription: A large data set containing about 2,9000,000 loan applications issued from 2007 to 2020Q3. These loans are categorized into two terms: 36 months and 60 months. First, we extracted a representative sample comprising 5-10% of the dataset in order to perform exploratory data analysis (EDA) and gain some insights into the data.


Randomly select 5% of the population as sample 


In [3]:
DATA_PATH = os.getcwd()+'/18-LendingClub/data' if 'LendingClub' not in os.getcwd() else os.getcwd()+'/data'
meta = pd.read_csv(DATA_PATH+'/LCDataDictionary.csv',index_col=['LoanStatNew'])

def extract_loan_sample(p):
    ''' p % of the lines  '''

    cols = meta.index.to_list()
    df = pd.read_csv(DATA_PATH+'/Loan_status_2007-2020Q3.gzip', skiprows=lambda i: i>0 and random.random() > p, low_memory=False)[cols]
    df['fico_score']  =  (df['fico_range_high']+df['fico_range_low'])/2.0
    df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
    df = df.drop(columns=['fico_range_high','fico_range_low','emp_title'])
    return df

sample = extract_loan_sample(p=0.05)

In [4]:
print ('Data sample size: ',len(sample), ', Issue date 2007 - 2020Q3')
logger.info('Column Description')
display(meta.T)
logger.info('Example of Data')
sample.head(3)


Data sample size:  146228 , Issue date 2007 - 2020Q3
INFO - - - - - - - - Column Description  - - - - - - - 


LoanStatNew,loan_amnt,loan_status,acc_now_delinq,addr_state,annual_inc,application_type,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,...,pub_rec,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,term,title,total_acc,verification_status
Description,The listed amount of the loan applied for by t...,Current status of the loan,The number of accounts on which the borrower i...,The state provided by the borrower in the loan...,The self-reported annual income provided by th...,Indicates whether the loan is an individual ap...,Number of charge-offs within 12 months,The number of 30+ days past-due incidences of ...,The past-due amount owed for the accounts on w...,debt to income ratio,...,Number of derogatory public records,Number of public record bankruptcies,A category provided by the borrower for the lo...,Total credit revolving balance,"Revolving line utilization rate, or the amount...",LC assigned loan subgrade,The number of payments on the loan. Values are...,The loan title provided by the borrower,The total number of credit lines currently in ...,"Indicates if income was verified by LC, not ve..."


INFO - - - - - - - - Example of Data  - - - - - - - 


,loan_amnt,loan_status,acc_now_delinq,addr_state,annual_inc,application_type,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,...,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,term,title,total_acc,verification_status,fico_score
0,10000,Fully Paid,0.0,TX,50000.0,Individual,0.0,0.0,0.0,11.18,...,0.0,credit_card,10056,82.4%,B3,36 months,Credit Card Loan,21.0,Source Verified,692.0
1,15000,Charged Off,0.0,NY,60000.0,Individual,0.0,0.0,0.0,15.22,...,0.0,debt_consolidation,5872,57.6%,C2,36 months,consolidation,11.0,Not Verified,697.0
2,5000,Charged Off,0.0,OH,100000.0,Individual,0.0,0.0,0.0,16.33,...,0.0,debt_consolidation,74351,62.1%,A5,36 months,Debt Consolidation for a better life,35.0,Source Verified,707.0


In [19]:
pd.DataFrame(data=sample.isnull().sum()/len(sample)*100,columns=['Percentage of Null']).plot.bar()

<a id='section2'></a>

### 2 Pre-processing and EDA:

We filter out loans that are not yet matured and treat "Fully Paid" as 0 and "Charged Off" or "Default" as 1.  
Here, we only consider loans with those two status, as "Current" also has potentially go default in near future.  We also do not consider the loan status of "Does not meet the credit policy". Those are the loans issued at the early stage of the company where the system is not fully developed. 
#### 2.1 Data Cleaning
- Only keep status of "Fully Paid  " or "Charged Off"

In [31]:
logger.info('Raw Data')
print ('sample size before cleaning: ', len(sample))
print (sample['loan_status'].value_counts(normalize=True))

INFO - - - - - - - - Raw Data  - - - - - - - 
sample size before cleaning:  146228
loan_status
Fully Paid                                             0.510655
Current                                                0.353961
Charged Off                                            0.123431
Late (31-120 days)                                     0.005567
In Grace Period                                        0.003795
Late (16-30 days)                                      0.000807
Issued                                                 0.000698
Does not meet the credit policy. Status:Fully Paid     0.000670
Does not meet the credit policy. Status:Charged Off    0.000253
Default                                                0.000164
Name: proportion, dtype: float64


In [41]:
def data_cleaning(df):
    df = df.copy()
    df = df.loc[df['loan_status'].isin(['Fully Paid','Charged Off'])]
    print ('sample size :', len(df))    
    fig = (df['loan_status'].value_counts(normalize=True)*100
          ).plot.bar(
                title="Loan status distribution after cleaning", 
                template="simple_white",
                labels=dict(value="Percentage", variable="option")
            )
    fig.update_layout(
        autosize=False,
        width=800,
        height=350,
    )
    fig.show()

    print (df['loan_status'].value_counts(normalize=True))
#     pd.DataFrame(data=df['loan_status'].value_counts(normalize=True),columns=['Percentage']).plot.bar()
#     plt.xticks(rotation=0)
    plt.show()
#     return df
    
data_cleaning(sample)

sample size : 92721


loan_status
Fully Paid     0.805341
Charged Off    0.194659
Name: proportion, dtype: float64


In [ ]:
#Types of loan_status
def data_filtering(df):
    df= df.copy()
    last_date = '2020-09-01'
    num_of_applications = len(df)
    print (df['loan_status'].value_counts(normalize=True))
    (df['loan_status'].value_counts(normalize=True)).plot.bar()
    plt.title('loan status')
#     plt.show()

    return df




    df['year'] = df.issue_d.dt.year    
    loan_status = groupby_factor(df,factor='year')
    print (loan_status,'\n')
    
    years = loan_status.index[2:-3]
    default_rate = loan_status['Default'][2:-3]

    # Perform Pearson correlation test
    correlation_coefficient, p_value = stats.pearsonr(years, default_rate)
    print ('correlation coeff = %.3f,  p_value=%.3f  '%(correlation_coefficient, p_value))
    # Set the significance level (alpha)
    alpha = 0.05
    if p_value < alpha:
        print("There is a statistically significant correlation between years and default rate.")
    else:
        print("There is no statistically significant correlation between years and default rate.")

    return df
  
print ('sample size: ',len(sample),'\n')
print (title(' loan status % '))
sample = data_filtering(sample)







#### 2.1 What could be affecting the loan default rate ? 
    - Has default rate increased/decreased over time ? 
    - Grades. A > B > C > D > E > F
    - Loan terms: Longer terms can increase default risk
    - How is income affecting default rate ? 
    - Debt-to-Income Ratio
    - Loan Amount
    ... More


##### 2.1.1 Hypothsis test
To test whether years and the default rate are correlated, a statistical test such as the Pearson correlation coefficient is used (also known as Pearson's r). Here we set the threshold alpha as 0.5
- if p value < alpha     :  significant correlation
- if p value > alpha     :  no correlation

In [ ]:

sample['loan_status'].value_counts()

In [ ]:
def data_featuring(df):
    df = df.loc[df['loan_status'].isin(['Fully Paid','Default'])]
    X = df['application_type'].values.reshape(-1, 1)
    enc = OneHotEncoder().fit(X)
#     print (enc.transform(X).toarray())
    
    print (df.title.unique())
    return df
_= data_featuring(sample)

In [ ]:
# verification_status	

In [ ]:
####     BY GRADE   ##########
def data_analysis(data):
    
    df = data[['term','grade','issue_d','loan_status','annual_inc']].copy()
    # -------    Grade    ----------
    grade = groupby_factor(df,factor='grade')
    print (title('grade'), grade,'\n')
    ax = grade.plot(kind='bar', rot=0)
    ax.set_ylabel('%')
    
    # -------    Term    ----------
    term = groupby_factor(df,factor='term')
    print (title('term'), term,'\n')
    
    # -------    Income class   ----------

    df['income_class'] = pd.cut(df.annual_inc,[1,50000,100000,500000,10000000,max(df.annual_inc.max(),10000001)])
    income_class = groupby_factor(df,factor='income_class')
    print (title('income_class'),income_class ,'\n')

    # -------    Has default rate increased/decreased over time   ----------
    df['year'] = df.issue_d.dt.year
    overtime = groupby_factor(df,factor='year')
    print (title('Over years'), overtime,'\n')

    
# print (df_loan.head(2).T)
print ('   IN   %    ')
data_analysis(data=sample)

We can safely neglect the beginning and ending years because it takes 3-5 years to determine the loan status (loans either take 3 or 5 years to mature). From the figure below, there is a clearly an upward trend in the default loan rate over the years. 

In [ ]:
def by_issue_time():
    df = default_rate_by(df_loan,by=['issue_d'])
    df = df.sort_values(by=['issue_d']).fillna(0)
    print ('data started :', df.index.min(),'    end :',df.index.max())

    sns.scatterplot(data=df, x="issue_d", y="Default Rate %").set_title('default rate')
    df = df.loc[(df.index>='2009-07-01')&(df.index<='2019-03-01')] #loan maturity 3-5 years
    sns.scatterplot(data=df, x="issue_d", y="Default Rate %").set_title('default rate')
    plt.legend(labels=["whole period","effective period"])

#     return df
by_issue_time()

#### 2.2 Factors 
    What factors may affect default rate ? 
    - 2). Default rate at different grade
    - 
Because we set charged off as 1, the positive correlation means the loan more likely to go default. From the correlation map below, we see that borrowers with delinq history, public records, bankcrupt history, high debt to income ratio (dti),higher bankcard utilization rate more likely to go default. Higher loan amount, multiple inqury attempts in 6 months will also increase the risk. Whereas borrowers with higher income, higher fico score, more credit cards in use tend to pay off the debt. 



#### 3 Modelling 
- 3.1 train-test split 
- Random Forest
- Logistic regression 




In [ ]:
plt.figure(figsize=(20, 8))
sns.heatmap(df_loan.corr(), annot=True, cmap='viridis')

In [ ]:
#classification
def default_rate_by(data, by='grade'):
    df = data.groupby([by,'loan_status']).agg({'count':'count'})
    df = pd.pivot_table(df, values='count', index=[by],columns=['loan_status'])
    df['Default Rate %'] = df[1]/df.sum(axis=1)*100
    return df.round(2)
default_rate_by(df_loan,by='grade')

reference: https://cs229.stanford.edu/proj2018/report/69.pdf    